In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics import root_mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import GradientBoostingRegressor
from IPython.display import display
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.model_selection import TimeSeriesSplit

In [ ]:
energy_train = pd.read_parquet('energy_train.parquet')
energy_test2 = pd.read_parquet('energy_test2.parquet')
energy_test1 = pd.read_parquet('energy_test1.parquet')
forecasts = pd.read_parquet('forecasts.parquet')

energy_test1_copy = pd.read_parquet('energy_test1.parquet')

### Zusammenführen der Wettermodelle

In [ ]:
for model in ['DWD ICON', 'NCEP GFS']:
    # Filter für das Wettermodell
    forecasts_model = forecasts[forecasts['Weather Model'] == model].copy()
    
    # Spalten umbenennen
    forecasts_model = forecasts_model.rename(columns={
        'SolarDownwardRadiation': f'SolarDownwardRadiation_{model.replace(" ", "_")}',
        'CloudCover': f'CloudCover_{model.replace(" ", "_")}',
        'Temperature': f'Temperature_{model.replace(" ", "_")}'
    })
    
    # 'valid_datetime' berechnen
    forecasts_model['valid_datetime'] = pd.to_datetime(forecasts_model['ref_datetime']) + pd.to_timedelta(forecasts_model['valid_time'], unit='h')
    
    # Forecast DataFrame für das spezifische Modell speichern
    if model == 'DWD ICON':
        forecasts_dwd = forecasts_model
    else:
        forecasts_ncep = forecasts_model

# Zusammenführen der beiden Modelle
forecasts_combined = pd.merge(
    forecasts_ncep,
    forecasts_dwd, 
    on=['ref_datetime', 'valid_time', 'valid_datetime'], 
    how='inner'
)

forecasts_combined.head()


### Zusammenführen der Energiedaten und Wettervorhersagen

In [ ]:
# Merge beide DataFrames basierend auf den Spalten 'dtm' und 'ref_datetime' in energy_train
# sowie 'valid_datetime' und 'ref_datetime' in forecasts_combined (inner join).
energy_train_mit_forecast = pd.merge(
    energy_train, 
    forecasts_combined, 
    left_on=['dtm', 'ref_datetime'], 
    right_on=['valid_datetime', 'ref_datetime'], 
    how='inner'
)

# Entferne Zeilen, bei denen die Zielvariable 'Solar_MWh' NaN ist.
energy_train_mit_forecast = energy_train_mit_forecast[energy_train_mit_forecast["Solar_MWh"].isna() == False]

energy_train_mit_forecast.head()


In [ ]:
# Extrahiere Monat und Jahr aus der Spalte 'dtm' im Format "Monat Jahr"
energy_train_mit_forecast['month_year'] = energy_train_mit_forecast['dtm'].dt.strftime('%B %Y')

# Erhalte eindeutige Werte der Monate und Jahre
unique_months_years = energy_train_mit_forecast['month_year'].unique()

print(unique_months_years)


### Zusammenführen der Testdaten und Wettervorhersagen

In [ ]:
# Merge energy_test1 und forecasts_combined basierend auf 'dtm' und 'valid_datetime' (left join)
energy_test1_mit_forecast = pd.merge(
    energy_test1,
    forecasts_combined,
    left_on='dtm',
    right_on='valid_datetime',
    how='left'
)

# Behalte nur Zeilen, bei denen 'ref_datetime_y' (forecasts_combined) <= 'ref_datetime_x' (energy_test1)
energy_test1_mit_forecast = energy_test1_mit_forecast[energy_test1_mit_forecast['ref_datetime_y'] <= energy_test1_mit_forecast['ref_datetime_x']]

# Für jede 'dtm'-Zeile: Behalte die Zeile mit dem neuesten 'ref_datetime_y' (höchster Wert)
energy_test1_mit_forecast = energy_test1_mit_forecast.loc[energy_test1_mit_forecast.groupby('dtm')['ref_datetime_y'].idxmax()]

# Anzahl der fehlenden Werte in 'valid_datetime' ausgeben
print(energy_test1_mit_forecast['valid_datetime'].isnull().sum())

# Übersicht der DataFrame-Struktur und Spalteninformationen anzeigen
energy_test1_mit_forecast.info()


### (Nur für Test 2) Finde die fehlenden Zeilen: 

In [ ]:
#Identifiziere die Zeilen in `energy_test1`, deren `dtm`-Werte nicht in der Spalte `dtm` von `energy_test1_mit_forecast` enthalten sind. 

#missing_rows = energy_test1[~energy_test1['dtm'].isin(energy_test1_mit_forecast['dtm'])]

#print(missing_rows)

In [ ]:
#energy_test1_copy = energy_test1_copy[energy_test1_copy['dtm'].isin(energy_test1_mit_forecast['dtm'])]

### Welche Daten werden von energy_test1_mit_forecast abgedeckt?

In [ ]:
# Extrahiere Monat und Jahr aus der Spalte 'dtm' im Format "Monat Jahr"
energy_test1_mit_forecast['month_year'] = energy_test1_mit_forecast['dtm'].dt.strftime('%B %Y')

# Erhalte eindeutige Werte der Monate und Jahre
unique_months_years = energy_test1_mit_forecast['month_year'].unique()

print(unique_months_years)

###  Generieren neuer Features(Train)

In [ ]:
energy_train_mit_forecast['time'] = energy_train_mit_forecast['dtm'].dt.time  #Stunden extrahieren
energy_train_mit_forecast['effective_radiation'] = energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON'] * (1 - energy_train_mit_forecast['CloudCover_DWD_ICON'])
energy_train_mit_forecast.head()


### Generieren neuer Features(Test)

In [ ]:
energy_test1_mit_forecast['time'] = energy_test1_mit_forecast['dtm'].dt.time  #Stunden extrahieren
energy_test1_mit_forecast['effective_radiation'] = energy_test1_mit_forecast['SolarDownwardRadiation_DWD_ICON'] * (1 - energy_test1_mit_forecast['CloudCover_DWD_ICON'])


### Behandlung von fehlenden Datensätzen

In [ ]:
# 1. Gruppiere die Daten nach der Spalte 'time'. Dies teilt die Daten in Gruppen basierend auf gleichen Zeitwerten.
# 2. Wende innerhalb jeder Gruppe eine "forward fill" (ffill) Methode an. Diese Methode füllt fehlende Werte
#    (NaN) durch den zuletzt bekannten Wert aus der vorhergehenden Zeile innerhalb der Gruppe.
energy_train_mit_forecast['SolarDownwardRadiation_DWD_ICON'] = (
    energy_train_mit_forecast.groupby('time')['SolarDownwardRadiation_DWD_ICON']
    .ffill()
)

energy_train_mit_forecast['SolarDownwardRadiation_NCEP_GFS'] = (
    energy_train_mit_forecast.groupby('time')['SolarDownwardRadiation_NCEP_GFS']
    .ffill()
)


In [ ]:
# 1. Gruppiere die Daten nach der Spalte 'time'. Dies teilt die Daten in Gruppen basierend auf gleichen Zeitwerten.
# 2. Wende innerhalb jeder Gruppe eine "forward fill" (ffill) Methode an. Diese Methode füllt fehlende Werte
#    (NaN) durch den zuletzt bekannten Wert aus der vorhergehenden Zeile innerhalb der Gruppe.
energy_test1_mit_forecast['SolarDownwardRadiation_DWD_ICON'] = (
    energy_test1_mit_forecast.groupby('time')['SolarDownwardRadiation_DWD_ICON']
    .ffill()
)

In [ ]:
#energy_train_mit_forecast = energy_train_mit_forecast[~energy_train_mit_forecast['month'].isin(['November', 'December'])]
#energy_test1_mit_forecast = energy_test1_mit_forecast[~energy_test1_mit_forecast['month'].isin(['November', 'December'])]

### Aufteilung des DataFrames in Tag- und Nachtzeiten basierend auf Solar_MWh und SolarDownwardRadiation

In [ ]:
# Bedingung: Nachtzeit 
night_condition = (energy_train_mit_forecast['Solar_MWh'] == 0) & (energy_train_mit_forecast['SolarDownwardRadiation_NCEP_GFS'] == 0)

# Daten für die Nachtzeit filtern
night = energy_train_mit_forecast[night_condition]

# Daten für die Tageszeit filtern (alles, was nicht Nacht ist)
day = energy_train_mit_forecast[~night_condition]


In [ ]:
# Bedingung: Nachtzeiten im Testdatensatz 
night_condition_test = (
    energy_test1_mit_forecast['SolarDownwardRadiation_NCEP_GFS'] == 0
)

# Filtere Daten für Nachtzeiten im Testdatensatz
night_test = energy_test1_mit_forecast[night_condition_test]

# Filtere Daten für Tageszeiten im Testdatensatz (alles, was nicht Nacht ist)
day_test = energy_test1_mit_forecast[~night_condition_test]

In [ ]:
df_train_night = night
df_train_day = day

In [ ]:
df_train_day.head()

### Zu entfernende Spalten

In [ ]:
# Liste der zu entfernenden Spalten(Training)
columns_to_drop = ["dtm", "ref_datetime", "Weather Model_x", "Weather Model_y", "valid_datetime", "valid_time", 'month_year',
                   "CloudCover_NCEP_GFS","CloudCover_DWD_ICON",  "Temperature_NCEP_GFS", "Solar_capacity_mwp", "SolarDownwardRadiation_NCEP_GFS", "Temperature_DWD_ICON"] 

# Entfernen der definierten Spalten(Training)
df_train_night = df_train_night.drop(columns=columns_to_drop)
df_train_day = df_train_day.drop(columns=columns_to_drop)


In [ ]:
df_train_night

In [ ]:
# Liste der zu entfernenden Spalten(Test)
columns_to_drop_test = ["dtm", "ref_datetime_x", "ref_datetime_y", "Weather Model_x", "Weather Model_y", "valid_datetime", "valid_time", "month_year",
                    "CloudCover_NCEP_GFS","CloudCover_DWD_ICON", "Temperature_NCEP_GFS", "SolarDownwardRadiation_NCEP_GFS", "Solar_capacity_mwp", "Temperature_DWD_ICON"]

# Entfernen der definierten Spalten(Test)
night_test = night_test.drop(columns=columns_to_drop_test)
day_test = day_test.drop(columns=columns_to_drop_test)

### Training-Test-Split

In [ ]:
X_predict_night = night_test
X_predict_day = day_test

In [ ]:
categorical_features = ["time"] 
numerical_features = ["SolarDownwardRadiation_DWD_ICON","effective_radiation"] 

In [ ]:
# Entfernen des Labels aus den Daten
y_train_night = df_train_night.pop("Solar_MWh")
y_train_day = df_train_day.pop("Solar_MWh")

In [ ]:
X_train_night = df_train_night
X_train_day = df_train_day

### Columntransformer and pipeline

In [ ]:
# Ziel: Vorverarbeitung der Daten (kategoriale und numerische Features) in separaten Pipelines.
column_trans = ColumnTransformer(
    transformers=[
        # 1. Vorverarbeitung der kategorialen Features:
        #    a) Fehlende Werte (NaN) in kategorialen Spalten durch die häufigste Kategorie ersetzen ("most_frequent").
        #    b) One-hot Encoding für kategoriale Features, um sie in numerische Werte zu transformieren.
        ("onehot", Pipeline([
            ("impute", SimpleImputer(strategy="most_frequent")),  # Fehlende Werte ersetzen
            ("encode", OneHotEncoder(handle_unknown="ignore"))  # One-hot Encoding
        ]), categorical_features),  # Liste der kategorialen Features
        
        # 2. Vorverarbeitung der numerischen Features:
        #    a) Skalierung der numerischen Spalten, um sie standardisiert (Mittelwert=0, Varianz=1) darzustellen.
        ("impute_scale", Pipeline([
            ("impute", SimpleImputer(strategy="mean")),  # Handle nulls in numerical
            ("scale", StandardScaler())  # Skalierung der numerischen Daten
        ]), numerical_features)  # Liste der numerischen Features
    ],
    # Alle anderen Spalten werden unverändert beibehalten (falls vorhanden), da `remainder="passthrough"` angegeben ist.
    remainder="passthrough"
)


In [ ]:
# 1. Wendet die oben definierte Vorverarbeitung (column_trans) auf die Daten an.
# 2. Nutzt einen GradientBoostingRegressor als Modell für die Vorhersage.
pipe = Pipeline([
    ("preprocessing", column_trans),
    ("model", GradientBoostingRegressor(random_state=42))
])

In [ ]:
# Hyperparametersuche für die Pipeline:
# 1. `learning_rate`, `n_estimators`, `max_depth` und `max_features` werden für den GradientBoostingRegressor getestet.
# 2. Die Strategie für das Imputing in der OneHot-Encoding-Pipeline wird auf "most_frequent" gesetzt.

param_grid = {
    "model__learning_rate": [0.01, 0.1, 0.2],
    "model__n_estimators": [100, 200, 500],
    "model__max_depth": [3, 5, 10],
    "model__max_features": ["sqrt", "log2", None],
    "preprocessing__onehot__impute__strategy": ["most_frequent"],  # Strategy for categorical imputation
    "preprocessing__impute_scale__impute__strategy": ["mean"]
}

In [ ]:
# TimeSeriesSplit für zeitabhängige Kreuzvalidierung:
# Ziel: Aufteilen der Daten in Trainings- und Testsets, wobei zukünftige Daten nicht zur Validierung von Modellen 
# auf vergangenen Daten verwendet werden. Hier wird in 3 Splits unterteilt.
tscv = TimeSeriesSplit(n_splits=3)

# Einrichtung von GridSearchCV:
# 1. `estimator`: Die Pipeline `pipe`, die Vorverarbeitung und Modellierung kombiniert.
# 2. `param_grid`: Hyperparameter, die getestet werden sollen.
# 3. `scoring`: Bewertungsmetrik (negativer RMSE, da RMSE minimiert werden soll; GridSearchCV maximiert standardmäßig).
# 4. `cv`: Die TimeSeriesSplit-Strategie für die Kreuzvalidierung.
# 5. `n_jobs=-1`: Nutzt alle verfügbaren CPU-Kerne für paralleles Training.
gs = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring="neg_root_mean_squared_error", cv=tscv, n_jobs=-1)

# Anpassung von GridSearchCV an die Trainingsdaten (nur für Nachtzeiten):
# Sucht die beste Kombination der Hyperparameter im `param_grid`, basierend auf den Trainingsdaten.
gs.fit(X_train_night, y_train_night)

# Abrufen der besten Parameter und des besten Modells:
# 1. `best_params_`: Die beste Kombination der Hyperparameter, die im GridSearchCV gefunden wurde.
# 2. `best_estimator_`: Das Modell (Pipeline) mit den optimalen Hyperparametern.
gs_best_params = gs.best_params_
gs_best_model_night = gs.best_estimator_

# Berechnung des RMSE (Root Mean Squared Error) aus dem besten CV-Score:
# `gs.best_score_` gibt den negativen RMSE zurück, daher wird dieser invertiert und die Wurzel gezogen.
train_rmse = np.sqrt(-gs.best_score_)

# Ausgabe der besten Parameter und der besten RMSE (Cross-Validation):
print("Best Parameters:", gs.best_params_)  # Zeigt die optimalen Hyperparameter an.
print("Best RMSE (Cross-Validation):", -gs.best_score_)  # Zeigt den negativen RMSE aus der besten CV-Kombination.


In [ ]:
# Vorhersage der Zielvariable 'Solar_MWh' für neue Daten:
# 1. `best_model_night`: Das aus GridSearchCV hervorgegangene beste Modell, das mit den Nachtzeit-Daten trainiert wurde.
# 2. `X_predict_night`: Der Datensatz mit den Eingabefeatures, für den Vorhersagen gemacht werden sollen.
# 3. `.predict`: Nutzt das trainierte Modell, um die Zielvariable 'Solar_MWh' basierend auf den Eingabefeatures vorherzusagen.

X_predict_night['Solar_MWh_pred'] = gs_best_model_night.predict(X_predict_night)

In [ ]:
X_predict_night

In [ ]:
# TimeSeriesSplit für zeitabhängige Kreuzvalidierung:
# Ziel: Die Daten werden in 3 Splits unterteilt, wobei frühere Daten nicht zur Validierung auf späteren Daten genutzt werden.
tscv = TimeSeriesSplit(n_splits=3)

# Einrichtung von GridSearchCV:
# 1. `estimator`: Die Pipeline `pipe`, die Vorverarbeitung und Modell kombiniert.
# 2. `param_grid`: Hyperparameter, die getestet werden sollen.
# 3. `scoring`: Bewertungsmetrik (negativer RMSE, da RMSE minimiert werden soll; GridSearchCV maximiert standardmäßig).
# 4. `cv`: Die TimeSeriesSplit-Strategie für die Kreuzvalidierung.
# 5. `n_jobs=-1`: Nutzt alle verfügbaren CPU-Kerne für paralleles Training.
gs = GridSearchCV(estimator=pipe, param_grid=param_grid, scoring="neg_root_mean_squared_error", cv=tscv, n_jobs=-1)

# Anpassung von GridSearchCV an die Tageszeit-Trainingsdaten:
# Sucht die beste Kombination der Hyperparameter im `param_grid`, basierend auf den Tageszeit-Trainingsdaten.
gs.fit(X_train_day, y_train_day)

# Abrufen der besten Parameter und des besten Modells:
# 1. `best_params_`: Die optimale Kombination der Hyperparameter, die während der GridSearch gefunden wurde.
# 2. `best_estimator_`: Das trainierte Modell (Pipeline) mit den optimalen Hyperparametern.
best_params = gs.best_params_
best_model_day = gs.best_estimator_

# Ausgabe der besten Parameter und des besten RMSE (Cross-Validation):
# `gs.best_score_` gibt den negativen RMSE zurück. Daher wird dieser invertiert für die Ausgabe.
print("Best Parameters:", gs.best_params_)  # Zeigt die optimalen Hyperparameter an.
print("Best RMSE (Cross-Validation):", -gs.best_score_)  # Zeigt den negativen RMSE aus der besten CV-Kombination.


In [ ]:
# Vorhersage der Zielvariable 'Solar_MWh' für neue Tageszeit-Daten:
# 1. `best_model_day`: Das aus GridSearchCV hervorgegangene beste Modell, das mit den Tageszeit-Daten trainiert wurde.
# 2. `X_predict_day`: Der Datensatz mit den Eingabefeatures, für den Vorhersagen gemacht werden sollen.
# 3. `.predict`: Nutzt das trainierte Modell, um die Zielvariable 'Solar_MWh' basierend auf den Eingabefeatures vorherzusagen.

X_predict_day['Solar_MWh_pred'] = best_model_day.predict(X_predict_day)

In [ ]:
# Kombiniere Vorhersagedaten:
# 1. `pd.concat`: Fügt die beiden DataFrames `X_predict_night` (Vorhersagen für Nachtzeiten) und
#    `X_predict_day` (Vorhersagen für Tageszeiten) entlang der Zeilen (axis=0) zusammen.
X_predict_combined = pd.concat([X_predict_night, X_predict_day], axis=0)

# Sortiere den kombinierten DataFrame nach dem ursprünglichen Index:
# Ziel: Sicherstellen, dass die Reihenfolge der Zeilen im kombinierten DataFrame
# mit der ursprünglichen Zeitachse oder dem ursprünglichen Index übereinstimmt.
X_predict_combined = X_predict_combined.sort_index()

In [ ]:
X_predict_combined.head(20)


In [ ]:
# Kopieren der Spalte 'Solar_MWh_pred' aus dem kombinierten Vorhersagedatensatz:
energy_test1_copy['Solar_MWh_pred'] = X_predict_combined['Solar_MWh_pred'].values

In [ ]:
energy_test1_copy.head()

### Speichern der DataFrames

#### Test 1

In [ ]:

energy_test1_copy.to_pickle("energy_test1predict.pkl")

#### Test 2

In [ ]:
#rejoined_data = pd.concat([energy_test1_copy, missing_rows])

In [ ]:
# SimpleImputer erstellen, um fehlende Werte (NaN) durch den Median zu ersetzen
#imp_median = SimpleImputer(missing_values=np.nan, strategy='median')

# Median berechnen und fehlende Werte in 'Solar_MWh_pred' ersetzen
#rejoined_data['Solar_MWh_pred'] = imp_median.fit_transform(rejoined_data[['Solar_MWh_pred']])

In [ ]:
#rejoined_data.to_pickle("energy_test2predict.pkl")